In [45]:
# Difference detector imports
import numpy as np
import cv2
# from matplotlib import pyplot as plt
from ipywidgets import *
from PIL import Image
from IPython.display import display, clear_output
import io
import pytesseract
from pytesseract import Output as py_Output

In [46]:
# Classifier Imports
from fastai.vision.all import *
from fastai.vision.widgets import *
import urllib.request

In [47]:
# Diff detector
MAX_FEATURES = 1000
GOOD_MATCH_PERCENT = 0.5

out_error = Output()

def alignImages(im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    matches = list(matches)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
#     cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
    
    out_error.clear_output()
    with out_error:
        if len(points1)<4 or len(points2)<4:
            print('The images are not similar enough for this POC, try other images please...')
        else:
            # Find homography
            h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

            # Use homography
            height, width, channels = im2.shape
            im1Reg = cv2.warpPerspective(im1, h, (width, height))

            return im1Reg 

In [48]:
# Diff detector
btn1_upload = FileUpload(description='After. Img')
btn2_upload = FileUpload(description='Before. Img')
out_pl_ref = Output()
out_pl_im = Output()
btn_load = Button(description='Show Images')

lbl_pred = widgets.Label()
confidence_pred = widgets.Label()

# Classifier
path = Path()
learn_inf = load_learner('mf_learner.pkl', cpu=True)

rotation = -0

def on_click_load(change):
    imReference = Image.open(io.BytesIO(btn1_upload.data[-1]))
    im = Image.open(io.BytesIO(btn2_upload.data[-1]))
    out_pl_ref.clear_output()
    out_pl_im.clear_output()
    out_pl_diff.clear_output()
    out_error.clear_output()
    with out_pl_ref:
        imReference.thumbnail([450,450])
        display(imReference.rotate(rotation))
    with out_pl_im:
        im.thumbnail([450,450])
        display(im.rotate(rotation))
        
btn_load.on_click(on_click_load)


btn_diff = Button(description='Show Difference')
out_pl_diff = Output()

def on_click_find_diff(change):
    try:
        imReference = np.array(Image.open(io.BytesIO(btn1_upload.data[-1])))
        im = np.array(Image.open(io.BytesIO(btn2_upload.data[-1])))

        og_img = imReference.copy()

        kernel = np.ones((6,6),np.uint8)
        blur_kern = (7,7)

        imReference = cv2.blur(imReference, blur_kern)
        imReference = cv2.morphologyEx(imReference, cv2.MORPH_OPEN, kernel)

        im = cv2.blur(im, blur_kern)
        im = cv2.morphologyEx(im, cv2.MORPH_OPEN, kernel)

        imReg = alignImages(im, imReference)

    #     remove black borders from aligned image
        black_border = np.where(imReg==0)
        imRef_crop = imReference.copy()
        imRef_crop[black_border]=0

    #     find difference between images
        diff = cv2.absdiff(imReg, imRef_crop)

        out_pl_diff.clear_output()
        with out_pl_diff:
            result = (diff > 75) * diff

            ##############
            # Grayscale
            gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

            gray = cv2.GaussianBlur(gray, (5, 5), 0)
            edged = cv2.Canny(gray, 50, 200)
            edged = cv2.dilate(edged, None, iterations=1)
            edged = cv2.erode(edged, None, iterations=1)

            # Finding Contours
            # Use a copy of the image e.g. edged.copy()
            # since findContours alters the image
            contours, hierarchy = cv2.findContours(edged, 
                cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            contours = sorted(contours, key=cv2.contourArea, reverse=True)

            if len(contours)==0:
                print("Error: please check the uploaded images...")
            else:
                num_contours = min(5, len(contours)) # draw n biggest contours on image
                similarity_score = np.zeros(num_contours)
                for cont in range(num_contours):
                    c = contours[cont]
                    x,y,w,h = cv2.boundingRect(c)
                    cv2.rectangle(og_img,(x,y),(x+w,y+h),(255,0,0),5) # red
                    similarity_score[cont] = w*h

                ################

    #                 # show the black image with diff highlighted
    #                 img2 = Image.fromarray(result)                
    #                 img2.thumbnail([450,450])
    #                 display(img2.rotate(rotation))

                # show the images
                img_contour = Image.fromarray(og_img) 
                img_contour.thumbnail([450,450])
                display(img_contour.rotate(rotation))

                size_of_image = np.shape(og_img)[0]*np.shape(og_img)[1]
                size_of_contours = similarity_score.sum()
                sim_score = size_of_contours/size_of_image*100

                # print similarity score
                print('Difference score: {:.1f}'.format(sim_score))

    #                 classifier
                img = PILImage.create(btn1_upload.data[-1])
                pred,pred_idx,probs = learn_inf.predict(img)
                if pred == 'Trench':
                    pred = 'Not yet fully restored (untidy)...'
                elif pred == 'Manhole':
                    pred = 'Seems as though a manhole is present; making classification difficult...'
                print(f'Prediction: {pred}; Probability: {probs[pred_idx]*100:.02f} %')
    except:
        pass


btn_diff.on_click(on_click_find_diff)

# Reinstatement Difference Detector

#### 1) Click on "After. Img" and select the final image after restoration
#### 2) Click on "Before. Img" and select the reference or neat image to which we need to restore the premises
#### 3) Click on "Show Images" to ensure the correct images have been selected
#### 4) Click on "Show Difference" to see which areas of the "After. Img" selected in Step 1 vary from the "Before. Img"

##### A difference score is also calculated. We can use this difference score to determine a pass/fail in future iterations

##### Note: This algorithm has been optimized based on images received from MetroFibre access builds. Using this detector on images of a different nature does not guarantee accurate results. Further hyper-parameter tuning is required to use different picture types. 

In [49]:
VBox([HBox([btn1_upload, btn2_upload, btn_load, btn_diff]), 
      VBox([HBox([out_pl_ref, out_pl_im]), out_error, out_pl_diff])])

In [51]:
# import session_info
# session_info.show()